### See `models/UBAR-MultiWOZ/ubar_preprocess.ipyn` for preprocessing. Esentially, we use the preprocessed data from UBAR

In [1]:
import os
import sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
	sys.path.insert(0, root_path)

In [8]:
import json

from datasets import load_dataset

In [27]:
dataset = load_dataset("multi_woz_v22", split="validation")

No config specified, defaulting to: multi_woz_v22/v2.2_active_only
Found cached dataset multi_woz_v22 (/home/xy2437/.cache/huggingface/datasets/multi_woz_v22/v2.2_active_only/2.2.0/7452f16a8b502e97df5c04cc4ee5436464762fa93b1ce778dd14181e79d8b51a)


In [13]:
sample_dialog = dataset[0]
sample_dialog

{'dialogue_id': 'MUL0484.json',
 'services': ['attraction', 'train'],
 'turns': {'turn_id': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
  'speaker': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
  'utterance': ['I need train reservations from norwich to cambridge',
   'I have 133 trains matching your request. Is there a specific day and time you would like to travel?',
   "I'd like to leave on Monday and arrive by 18:00.",
   'There are 12 trains for the day and time you request. Would you like to book it now?',
   'Before booking, I would also like to know the travel time, price, and departure time please.',
   'There are 12 trains meeting your needs with the first leaving at 05:16 and the last one leaving at 16:16. Do you want to book one of these?',
   'No hold off on booking for now. Can you help me find an attraction called cineworld cinema?',
   'Yes it is a cinema located in the south part of town what information would you like on it?',
   'Yes, that was all I needed. Thank you very 

In [14]:
len(sample_dialog['turns']['utterance']), len(sample_dialog['turns']['frames'])

(10, 10)

In [15]:
sample_dialog['turns']['frames'][0], sample_dialog['turns']['frames'][0]['state']

({'service': ['train'],
  'state': [{'active_intent': 'find_train',
    'requested_slots': [],
    'slots_values': {'slots_values_name': ['train-departure',
      'train-destination'],
     'slots_values_list': [['norwich'], ['cambridge']]}}],
  'slots': [{'slot': [],
    'value': [],
    'start': [],
    'exclusive_end': [],
    'copy_from': [],
    'copy_from_value': []}]},
 [{'active_intent': 'find_train',
   'requested_slots': [],
   'slots_values': {'slots_values_name': ['train-departure',
     'train-destination'],
    'slots_values_list': [['norwich'], ['cambridge']]}}])

In [16]:
for u, s in zip(sample_dialog['turns']['utterance'], sample_dialog['turns']['frames']):
	print(u)
	print(json.dumps(s, indent=4))

I need train reservations from norwich to cambridge
{
    "service": [
        "train"
    ],
    "state": [
        {
            "active_intent": "find_train",
            "requested_slots": [],
            "slots_values": {
                "slots_values_name": [
                    "train-departure",
                    "train-destination"
                ],
                "slots_values_list": [
                    [
                        "norwich"
                    ],
                    [
                        "cambridge"
                    ]
                ]
            }
        }
    ],
    "slots": [
        {
            "slot": [],
            "value": [],
            "start": [],
            "exclusive_end": [],
            "copy_from": [],
            "copy_from_value": []
        }
    ]
}
I have 133 trains matching your request. Is there a specific day and time you would like to travel?
{
    "service": [],
    "state": [],
    "slots": []
}
I'd like to leave on

In [32]:
dids = []
dataset = load_dataset("multi_woz_v22", split="test")

for x in dataset:
	dids.append(x['dialogue_id'])

# write to file
with open('../dataset/multiwoz/data/multi-woz/testListFile.json', 'w') as f:
	# write an id per line
	for did in dids:
		f.write(did + '\n') 

No config specified, defaulting to: multi_woz_v22/v2.2_active_only
Found cached dataset multi_woz_v22 (/home/xy2437/.cache/huggingface/datasets/multi_woz_v22/v2.2_active_only/2.2.0/7452f16a8b502e97df5c04cc4ee5436464762fa93b1ce778dd14181e79d8b51a)


In [33]:
dacts = None
with open('../dataset/multiwoz/data/multi-woz/dialogue_acts.json', 'r') as f:
	dacts = json.load(f)

In [34]:
new_dacts = {}
for k, v in dacts.items():
	new_dacts[k] = {}
	for turn, dact in v.items():
		new_dacts[k][turn] = dact['dialog_act']

In [36]:
with open('../dataset/multiwoz/data/multi-woz/dialogue_acts.json', 'w') as f:
	json.dump(new_dacts, f, indent=4)

In [37]:
delexicalized_dialogs = []
with open('../dataset/multiwoz/data/multi-woz/delex.json', 'r') as f:
	delexicalized_dialogs = json.load(f)

In [41]:
delexicalized_dialogs['SNG01856.json']

{'goal': {'taxi': {},
  'police': {},
  'hospital': {},
  'hotel': {'info': {'type': 'hotel',
    'parking': 'yes',
    'pricerange': 'cheap',
    'internet': 'yes'},
   'fail_info': {},
   'book': {'pre_invalid': True,
    'stay': '2',
    'day': 'tuesday',
    'invalid': False,
    'people': '6'},
   'fail_book': {'stay': '3'}},
  'topic': {'taxi': False,
   'police': False,
   'restaurant': False,
   'hospital': False,
   'hotel': False,
   'general': False,
   'attraction': False,
   'train': False,
   'booking': False},
  'attraction': {},
  'train': {},
  'message': ["You are looking for a <span class='emphasis'>place to stay</span>. The hotel should be in the <span class='emphasis'>cheap</span> price range and should be in the type of <span class='emphasis'>hotel</span>",
   "The hotel should <span class='emphasis'>include free parking</span> and should <span class='emphasis'>include free wifi</span>",
   "Once you find the <span class='emphasis'>hotel</span> you want to book it

In [40]:
delexicalized_dialogs['SNG01856.json']['log']

[{'text': 'am looking for a place to to stay that has [value_pricerange] price range it should be in a type of hotel',
  'metadata': {},
  'db_pointer': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   1.0,
   0.0,
   1.0,
   0.0,
   1.0,
   0.0]},
 {'text': 'okay , do you have a specific area you want to stay in ?',
  'metadata': {'taxi': {'book': {'booked': []},
    'semi': {'leaveAt': '',
     'destination': '',
     'departure': '',
     'arriveBy': ''}},
   'police': {'book': {'booked': []}, 'semi': {}},
   'restaurant': {'book': {'booked': [], 'time': '', 'day': '', 'people': ''},
    'semi': {'food': '', 'pricerange': '', 'name': '', 'area': ''}},
   'hospital': {'book': {'booked': []}, 'semi': {'department': ''}},
   'hotel': {'book': {'booked': [], 'stay': '', 'day': '', 'people': ''},
    'semi': {'name': 'not mentioned',
     'area':

## Tmp: UBAR

In [42]:
root_path = os.path.abspath(os.path.join('../models/UBAR-MultiWOZ'))
if root_path not in sys.path:
	sys.path.insert(1, root_path)

In [43]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Model
from reader import MultiWozReader

In [45]:
from argparse import Namespace


cfg = Namespace()

In [48]:
cfg.dbs = {
	'attraction': '../models/UBAR-MultiWOZ/db/attraction_db_processed.json',
	'hospital': '../models/UBAR-MultiWOZ/db/hospital_db_processed.json',
	'hotel': '../models/UBAR-MultiWOZ/db/hotel_db_processed.json',
	'police': '../models/UBAR-MultiWOZ/db/police_db_processed.json',
	'restaurant': '../models/UBAR-MultiWOZ/db/restaurant_db_processed.json',
	'taxi': '../models/UBAR-MultiWOZ/db/taxi_db_processed.json',
	'train': '../models/UBAR-MultiWOZ/db/train_db_processed.json',
}

In [50]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

# initialize multiwoz reader
reader = MultiWozReader(tokenizer)


KeyboardInterrupt



# Tmp: Galaxy

In [ ]:
root_path = os.path.abspath(os.path.join('../models/galaxy'))
if root_path not in sys.path:
	sys.path.insert(1, root_path)

In [7]:
import argparse
import json
import os
import random

import numpy as np
import torch

from galaxy.galaxy.data.field import MultiWOZBPETextField
from galaxy.args import parse_args
from galaxy.args import str2bool
from galaxy.data.dataset import Dataset
from galaxy.data.field import BPETextField, MultiWOZBPETextField, CamRestBPETextField, KvretBPETextField
from galaxy.trainers.trainer import Trainer, MultiWOZTrainer, CamRestTrainer, KvretTrainer
from galaxy.models.model_base import ModelBase
from galaxy.models.generator import Generator
from galaxy.utils.eval import MultiWOZEvaluator, CamRestEvaluator, KvretEvaluator

In [8]:
hparams_json = {
  "do_train": False,
  "do_test": False,
  "do_infer": True,
  "num_infer_batches": None,
  "hparams_file": None,
  # "BPETextField": {
    "vocab_path": "/data/nt12_ssd_gluster/myself/GALAXY/model/Bert/vocab.txt",
    "version": "2.1",
    "data_root": "/data/nt12_ssd_gluster/myself/GALAXY",
    "data_processed": "data_for_galaxy_encoded.data.json",
    "filtered": False,
    "max_len": 1024,
    "min_utt_len": 1,
    "max_utt_len": 50,
    "min_ctx_turn": 1,
    "max_ctx_turn": 20,
    "tokenizer_type": "Bert",
  # },
  # "Dataset": {
    "data_dir": None,
    "data_name": "multiwoz",
  # },
  # "Trainer": {
    "seed": 10,
    "gpu": 1,
    "valid_metric_name": "-loss",
    "num_epochs": 10,
    "save_dir": "/data/nt12_hdd_gluster/myself/GALAXY/outputs/multiwoz2.1/110-76.infer",
    "token_loss": True,
    "batch_size": 1,
    "log_steps": 100,
    "valid_steps": 2000,
    "save_checkpoint": True,
    "shuffle": True,
    "sort_pool_size": 0,
  # },
  # "Model": {
    "init_checkpoint": "/data/nt12_hdd_gluster/myself/GALAXY/outputs/multiwoz2.1/110-76/state_epoch_5",
    "model": "UnifiedTransformer",
    "num_token_embeddings": -1,
    "num_pos_embeddings": 512,
    "num_type_embeddings": 2,
    "num_turn_embeddings": 16,
    "num_act": 20,
    "num_heads": 12,
    "num_layers": 12,
    "hidden_dim": 768,
    "padding_idx": 0,
    "dropout": 0.1,
    "embed_dropout": 0.0,
    "attn_dropout": 0.1,
    "ff_dropout": 0.1,
    "use_discriminator": False,
    "dis_ratio": 1.0,
    "bce_ratio": 1.0,
    "pos_trainable": True,
    "with_joint_act": True,
    "with_rdrop_act": False,
    "initializer_range": 0.02,
    "lr": 0.0001,
    "weight_decay": 0.0,
    "gradient_accumulation_steps": 1,
    "warmup_steps": -1,
    "max_grad_norm": 5.0,
  # },
  # "Generator": {
    "generator": "BeamSearch",
    "min_gen_len": 1,
    "max_gen_len": 30,
    "use_true_prev_bspn": False,
    "use_true_prev_aspn": False,
    "use_true_db_pointer": False,
    "use_true_prev_resp": False,
    "use_true_curr_bspn": False,
    "use_true_curr_aspn": False,
    "use_all_previous_context": True,
    "use_true_bspn_for_ctr_eval": True,
    "use_true_domain_for_ctr_eval": True,
    "beam_size": 1,
    "length_average": False,
    "length_penalty": -1.0,
    "ignore_unk": True,
  # },
  "use_gpu": True
}

In [9]:
import json
from types import SimpleNamespace

hparams = json.loads(json.dumps(hparams_json), object_hook=lambda d: SimpleNamespace(**d))

In [10]:
bpe = MultiWOZBPETextField(hparams)

FileNotFoundError: [Errno 2] No such file or directory: '/data/nt12_ssd_gluster/myself/GALAXY/data/multiwoz2.1/vocab.freq.json'

In [ ]:
hparams.use_true_prev_bspn

AttributeError: 'types.SimpleNamespace' object has no attribute 'use_true_prev_bspn'